# LIBRERIAS

In [2]:
import pandas as pd
import os

# LECTURA

In [3]:
path = os.getcwd()
file = "VISA Reglas Intercambio V25.xlsx"
file_path = os.path.join(path, file).replace("\\", "/")

In [5]:
df = pd.read_excel(file_path, skiprows=3)

# Preprocesamiento

In [ ]:
## NO USAR ##
# Conversion del tipo de datos "object" a "str" 
columns_object = df.select_dtypes(include="object").columns
df[columns_object] = df[columns_object].astype(str)

In [6]:
# Creación de diccionario para jurisdicciones
jurisdiction_dictionary = (
    df.drop_duplicates(subset="REGION_COUNTRY_CODE")
    .set_index("REGION_COUNTRY_CODE")["JURISDICTION"]
    .to_dict()
)

jurisdiction_dictionary = {
    str(k): str(v) for k, v in jurisdiction_dictionary.items() if not pd.isna(k)
}
jurisdiction_dictionary


{'TT': 'Trinidad and Tobago',
 'RO': 'Romania',
 'PE': 'Peru',
 '3': 'LAC Intraregional',
 '9': 'Interregional',
 'HU': 'Hungary',
 'HN': 'Honduras',
 'GR': 'Greece',
 '5': 'Europe Intraregional',
 'DO': 'Dominican Republic',
 'KY': 'Cayman Islands',
 'BS': 'Bahamas',
 'AW': 'Aruba',
 'CW,SX': 'Dutch Area',
 'BB': 'Barbados',
 'ZA': 'South Africa',
 '7': 'CEMEA Intraregional'}

In [7]:
jurisdiction_count = int(input("Ingresar cantidad de jurisdicciones a generar: "))
jurisdiction_values = []

for i in range(0,jurisdiction_count):
    value = str(input(f"Ingresar jurisdiccion #'{i+1}: "))
    jurisdiction_values.append(value.upper())

print(jurisdiction_values)

['RO']


In [8]:
df_filter = df[(df['REGION_COUNTRY_CODE'].isin(jurisdiction_values)) & (df['VALID_UNTIL'].isnull())]

In [9]:
df_filter.head(5)

,JURISDICTION,REGION_COUNTRY_CODE,GUIDE_DATE,VALID_FROM,VALID_UNTIL,FEE_PROGRAM,INTELICA_ID,FPI,FEE_DESCRIPTOR,FEE_DESCRIPTION,...,SUMMARY_COMMODITY,PROCESSING_CODE_TRANSACTION_TYPE,POINT_OF_SERVICE_CONDITION_CODE,MESSAGE_IDENTIFIER,VALIDATION_CODE,V_I_P_FULL_FINANCIAL_MESSAGE_SETS,SENDER_DATA,ADDITIONAL_SENDER_DATA,SETTLEMENT_SERVICE,OTHER_CRITERIA_APPLIES
21812,Romania,RO,2024-08-01,2024-11-09,NaT,Romania Consumer Visa Workplace Benefits Card,1.0,?,RO EMP BENEFIT,Visa Workplace Benefits,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NSS,ISS",NaN
21813,Romania,RO,2024-08-01,2024-11-09,NaT,Romania Consumer Cards,2.0,?,RO MEAL VOUCH PP,All transactions with Meal Voucher subtype qua...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NSS,NaN
21814,Romania,RO,2024-08-01,2024-11-09,NaT,Romania Consumer Cards,3.0,?,RO CONT CR,Contactless Transaction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NSS,NaN
21815,Romania,RO,2024-08-01,2024-11-09,NaT,Romania Consumer Cards,4.0,?,RO CONT DB,Contactless Transaction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NSS,NaN
21816,Romania,RO,2024-08-01,2024-11-09,NaT,Romania Consumer Cards,5.0,?,RO SECURE CR,"EMV Terminal online authorised, magnetic strip...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NSS,NaN


In [10]:
def create_joins(column: str, value: str):
    match column:
        case "BUSINESS_MODE":
            print("hola")

In [57]:
from logging import handlers


def parse_operator_unique(value: str):
    if value.startswith("NOT:"):
        return "<>", value[4:].strip()
    return "=", value.strip()


def parse_operator_multiple(value: str):
    if value.startswith("NOT:"):
        return "NOT IN", value[4:].strip()
    return "IN", value.strip()


def split_str_values(value: str):
    return ",".join([f"'{v.strip()}'" for v in value.split(",")])


def split_int_values(value: str):
    return ",".join([f"{v.strip()}" for v in value.split(",")])


def replace_space(value: str):
    if value == "Space":
        return " "
    return value


def create_where(column: str, value: str):
    def business_mode(value):
        if value == "Acquiring":
            return "T1.[VSS_BMOD_ID] = 1"
        elif value == "Issuing":
            return "T1.[VSS_BMOD_ID] = 2"

    def issuer_country(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"CTRY.CTRY_DESC {operator} ({split_values})"

    def issuer_region(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"CTRY.CTRY_DESC {operator} ({split_values})"

    def technology_indicator(value):
        return "Hola"

    def product_id(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"P.BIN_PRD_CD {operator} ({split_values})"

    def fast_funds(value):
        operator, clean_value = parse_operator_unique(value)
        return f"FAST_FUND {operator} '{clean_value}'"

    def travel_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        return f"TRAVEL_INDICATOR {operator} '{clean_value}'"

    def b2b_program_id(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"B2B_PROGRAM_ID {operator} ({split_values})"

    def account_funding_source(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"FUNDING_SOURCE {operator} ({split_values})"

    def nnss_indicator(value):
        operator, clean_value = parse_operator_unique(value)
        replaced_value = replace_space(clean_value)
        return f"NNSS_INDICATOR {operator} '{replaced_value}'"

    def product_subtype(value):
        operator, clean_value = parse_operator_unique(value)
        return f"PRODUCT_SUBTYPE {operator} '{clean_value}'"

    def transaction_code(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_str_values(clean_value)
        return f"TRANSACTION_CODE {operator} ({split_values})"

    def transaction_code_qualifier(value):
        operator, clean_value = parse_operator_multiple(value)
        split_values = split_int_values(clean_value)
        return f"TRANSACTION_CODE_QUALIFIER {operator} ({split_values})"

    def issuer_bin_8(value):
        return "Hola"

    def acquirer_bin(value):
        return "Hola"

    def acquirer_business_id(value):
        return "Hola"

    def transaction_amount_currency(value):
        return "Hola"

    def transaction_amount(value):
        return "Hola"

    def acquirer_country(value):
        return "Hola"

    def acquirer_region(value):
        return "Hola"

    def merchant_country_code(value):
        return "Hola"

    def merchant_country_region(value):
        return "Hola"

    def merchant_category_code(value):
        return "Hola"

    def requested_payment_service(value):
        return "Hola"

    def usage_code(value):
        return "Hola"

    def authorization_characteristics_indicator(value):
        return "Hola"

    def authorization_code(value):
        return "Hola"

    def pos_terminal_capability(value):
        return "Hola"

    def cardholder_id_method(value):
        return "Hola"

    def pos_entry_mode(value):
        return "Hola"

    def timeliness(value):
        return "Hola"

    def reimbursement_attribute(value):
        return "Hola"

    def special_condition_indicator(value):
        return "Hola"

    def fee_program_indicator(value):
        return "Hola"

    def moto_eci_indicator(value):
        return "Hola"

    def acceptance_terminal_indicator(value):
        return "Hola"

    def prepaid_card_indicator(value):
        return "Hola"

    def pos_environment_code(value):
        return "Hola"

    def business_format_code(value):
        return "Hola"

    def business_application_id(value):
        return "Hola"

    def type_purchase(value):
        return "Hola"

    def network_identification_code(value):
        return "Hola"

    def message_reason_code(value):
        return "Hola"

    def surcharge_amount(value):
        return "Hola"

    def authorized_amount(value):
        return "Hola"

    def authorization_response_code(value):
        return "Hola"

    def merchant_verification_value(value):
        return "Hola"

    def dynamic_currency_indicator(value):
        return "Hola"

    def cvv2_result_code(value):
        return "Hola"

    def national_tax_indicator(value):
        return "Hola"

    def merchant_vat(value):
        return "Hola"

    def summary_commodity(value):
        return "Hola"

    def processing_code_transaction_type(value):
        return "Hola"

    def point_of_service_condition_code(value):
        return "Hola"

    handlers = {
        "BUSINESS_MODE": business_mode,
        "ISSUER_COUNTRY": issuer_country,
        "ISSUER_REGION": issuer_region,
        "TECHNOLOGY_INDICATOR": technology_indicator,
        "PRODUCT_ID": product_id,
        "FAST_FUNDS": fast_funds,
        "TRAVEL_INDICATOR": travel_indicator,
        "B2B_PROGRAM_ID": b2b_program_id,
        "ACCOUNT_FUNDING_SOURCE": account_funding_source,
        "NNSS_INDICATOR": nnss_indicator,
        "PRODUCT_SUBTYPE": product_subtype,
        "TRANSACTION_CODE": transaction_code,
        "TRANSACTION_CODE_QUALIFIER": transaction_code_qualifier,
        "ISSUER_BIN_8": issuer_bin_8,
        "ACQUIRER_BIN": acquirer_bin,
        "ACQUIRER_BUSINESS_ID": acquirer_business_id,
        "TRANSACTION_AMOUNT_CURRENCY": transaction_amount_currency,
        "TRANSACTION_AMOUNT": transaction_amount,
        "ACQUIRER_COUNTRY": acquirer_country,
        "ACQUIRER_REGION": acquirer_region,
        "MERCHANT_COUNTRY_CODE": merchant_country_code,
        "MERCHANT_COUNTRY_REGION": merchant_country_region,
        "MERCHANT_CATEGORY_CODE": merchant_category_code,
        "REQUESTED_PAYMENT_SERVICE": requested_payment_service,
        "USAGE_CODE": usage_code,
        "AUTHORIZATION_CHARACTERISTICS_INDICATOR": authorization_characteristics_indicator,
        "AUTHORIZATION_CODE": authorization_code,
        "POS_TERMINAL_CAPABILITY": pos_terminal_capability,
        "CARDHOLDER_ID_METHOD": cardholder_id_method,
        "POS_ENTRY_MODE": pos_entry_mode,
        "TIMELINESS": timeliness,
        "REIMBURSEMENT_ATTRIBUTE": reimbursement_attribute,
        "SPECIAL_CONDITION_INDICATOR": special_condition_indicator,
        "FEE_PROGRAM_INDICATOR": fee_program_indicator,
        "MOTO_ECI_INDICATOR": moto_eci_indicator,
        "ACCEPTANCE_TERMINAL_INDICATOR": acceptance_terminal_indicator,
        "PREPAID_CARD_INDICATOR": prepaid_card_indicator,
        "POS_ENVIRONMENT_CODE": pos_environment_code,
        "BUSINESS_FORMAT_CODE": business_format_code,
        "BUSINESS_APPLICATION_ID": business_application_id,
        "TYPE_PURCHASE": type_purchase,
        "NETWORK_IDENTIFICATION_CODE": network_identification_code,
        "MESSAGE_REASON_CODE": message_reason_code,
        "SURCHARGE_AMOUNT": surcharge_amount,
        "AUTHORIZED_AMOUNT": authorized_amount,
        "AUTHORIZATION_RESPONSE_CODE": authorization_response_code,
        "MERCHANT_VERIFICATION_VALUE": merchant_verification_value,
        "DYNAMIC_CURRENCY_INDICATOR": dynamic_currency_indicator,
        "CVV2_RESULT_CODE": cvv2_result_code,
        "NATIONAL_TAX_INDICATOR": national_tax_indicator,
        "MERCHANT_VAT": merchant_vat,
        "SUMMARY_COMMODITY": summary_commodity,
        "PROCESSING_CODE_TRANSACTION_TYPE": processing_code_transaction_type,
        "POINT_OF_SERVICE_CONDITION_CODE": point_of_service_condition_code,
    }

    if column in handlers:
        return handlers[column](value)
    else:
        raise KeyError(f"Unsupported column: {column}")


In [59]:
create_where("TRANSACTION_CODE_QUALIFIER","0,2")
#parse_operator("NOT:HN")

'TRANSACTION_CODE_QUALIFIER IN (0,2)'

In [12]:
from pandas import DataFrame


def generate_updates(df: DataFrame, table_name: str):
    query_updates = []
    columns = [df.columns[i] for i in range(17, 71)]

    for _, row in df.iterrows():
        conditions = []
        joins = []
        for column in columns:
            row_value = row[column]
            if pd.isna(row_value):
                pass
            else:
                match column:
                    case "BUSINESS_MODE":
                        if row_value == "Acquiring":
                            conditions.append(f"{column} ='{row_value}'")

        update_value = int(row["FEE_DESCRIPTOR"])
        query = (
            f"UPDATE T1 SET REI_CRI_ID = '{update_value}' FROM {table_name} T1 "
            + " LEFT JOIN ".join(joins)
            + "WHERE "
            + " AND ".join(conditions)
            + ";"
        )
        query_updates.append(query)

    return query_updates

In [ ]:
legacy_column_rename = {
    "BUSINESS_MODE": "VSS_BMOD_ID",
    "ISSUER_COUNTRY": "ARG_CTRY_ID",
    "ISSUER_REGION": "ARG_REG_ID",
    "TECHNOLOGY_INDICATOR": "",
    "PRODUCT_ID": "",
    "FAST_FUNDS": "",
    "TRAVEL_INDICATOR": "",
    "B2B_PROGRAM_ID": "",
    "ACCOUNT_FUNDING_SOURCE": "",
    "NNSS_INDICATOR": "",
    "PRODUCT_SUBTYPE": "",
    "TRANSACTION_CODE": "",
    "TRANSACTION_CODE_QUALIFIER": "",
    "ISSUER_BIN_8": "",
    "ACQUIRER_BIN": "",
    "ACQUIRER_BUSINESS_ID": "",
    "TRANSACTION_AMOUNT_CURRENCY": "",
    "TRANSACTION_AMOUNT": "",
    "ACQUIRER_COUNTRY": "",
    "ACQUIRER_REGION": "",
    "MERCHANT_COUNTRY_CODE": "",
    "MERCHANT_COUNTRY_REGION": "",
    "MERCHANT_CATEOGORY_CODE": "",
    "REQUESTED_PAYMENT_SERVICE": "",
    "USAGE_CODE": "",
    "AUTHORIZATION_CHARACTERISTICS_INDICATOR": "",
    "AUTHORIZATION_CODE": "",
    "POS_TERMINAL_CAPABILITY": "",
    "CARDHOLDER_ID_METHOD": "",
    "POS_ENTRE_MODE": "",
    "TIMELINESS": "",
    "REIMBURSEMENT_ATTRIBUTE": "",
    "SPECIAL_CONDITION_INDICATOR": "",
    "FEE_PROGRAM_INDICATOR": "",
    "MOTO_ECI_INDICATOR": "",
    "ACCEPTANCE_TERMINAL_INDICATOR": "",
    "PREPAID_CARD_INDICATOR": "",
    "POS_ENVIRONMENT_CODE": "",
    "BUSINESS_FORMAT_CODE": "",
    "BUSINESS_APPLICATION_ID": "",
    "TYPE_PURCHASE": "",
    "NETWORK_IDENTIFICATION_CODE": "",
    "MESSAGE_REASON_CODE": "",
    "SURCHARGE_AMOUNT": "",
    "AUTHORIZED_AMOUNT": "",
    "AUTHORIZATION_RESPONSE_CODE": "",
    "MERCHANT_VERIFICATION_VALUE": "",
    "DYNAMIC_CURRENCY_INDICATOR": "",
    "CVV2_RESULT_CODE": "",
    "NATIONAL_TAX_INDICATOR": "",
    "MERCHANT_VAT": "",
    "SUMMARY_COMMODITY": "",
    "PROCESSING_CODE_TRANSACTION_TYPE": "",
    "POINT_OF_SERVICE_CONDITION_CODE": "",
}

In [ ]:
columnas = [(index, col_name) for index, col_name in enumerate(df_filter.columns)]
columnas

[(0, 'JURISDICTION'),
 (1, 'REGION_COUNTRY_CODE'),
 (2, 'GUIDE_DATE'),
 (3, 'VALID_FROM'),
 (4, 'VALID_UNTIL'),
 (5, 'FEE_PROGRAM'),
 (6, 'INTELICA_ID'),
 (7, 'FPI'),
 (8, 'FEE_DESCRIPTOR'),
 (9, 'FEE_DESCRIPTION'),
 (10, 'COD_HIERARCHY'),
 (11, 'PROGRAM_DEFAULT'),
 (12, 'FEE_CURRENCY'),
 (13, 'FEE_VARIABLE'),
 (14, 'FEE_FIXED'),
 (15, 'FEE_MIN'),
 (16, 'FEE_CAP'),
 (17, 'BUSINESS_MODE'),
 (18, 'ISSUER_COUNTRY'),
 (19, 'ISSUER_REGION'),
 (20, 'TECHNOLOGY_INDICATOR'),
 (21, 'PRODUCT_ID'),
 (22, 'FAST_FUNDS'),
 (23, 'TRAVEL_INDICATOR'),
 (24, 'B2B_PROGRAM_ID'),
 (25, 'ACCOUNT_FUNDING_SOURCE'),
 (26, 'NNSS_INDICATOR'),
 (27, 'PRODUCT_SUBTYPE'),
 (28, 'TRANSACTION_CODE'),
 (29, 'TRANSACTION_CODE_QUALIFIER'),
 (30, 'ISSUER_BIN_8'),
 (31, 'ACQUIRER_BIN'),
 (32, 'ACQUIRER_BUSINESS_ID'),
 (33, 'TRANSACTION_AMOUNT_CURRENCY'),
 (34, 'TRANSACTION_AMOUNT'),
 (35, 'ACQUIRER_COUNTRY'),
 (36, 'ACQUIRER_REGION'),
 (37, 'MERCHANT_COUNTRY_CODE'),
 (38, 'MERCHANT_COUNTRY_REGION'),
 (39, 'MERCHANT_CATEGOR